In [ ]:
!pip install nest_asyncio

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import joblib

# Train model
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "models/diabetes_model_lr.pkl")

In [5]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import joblib

# Prepare app and model
app = FastAPI()
model = joblib.load("models/diabetes_model_lr.pkl")

# Define input schema
class DiabetesInput(BaseModel):
    data: list  # List of 10 features

@app.post("/predict")
def predict(input: DiabetesInput):
    input_data = np.array([input.data])
    prediction = model.predict(input_data)
    return {"prediction": float(prediction[0])}

In [7]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()  # Needed for running Uvicorn inside notebook

# Run the app (non-blocking)
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [62538]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [62538]


In [ ]:
import requests

data = {
    "data": [0.038, 0.050, 0.061, 0.022, -0.044, -0.034, -0.043, -0.002, 0.019, -0.017]
}

response = requests.post("http://127.0.0.1:8000/predict", json=data)
print("Prediction result:", response.json())